In [126]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

In [127]:
# Load the data
file_path = Path("../Resources/Source_data/FW_Veg_Rem_Combined.csv")
fires_df = pd.read_csv(file_path)
fires_df = fires_df[["fire_size","fire_size_class","state","Vegetation","Temp_pre_30","Temp_pre_15","Temp_pre_7","Temp_cont","Wind_pre_30","Wind_pre_15","Wind_pre_7","Wind_cont","Hum_pre_30","Hum_pre_15","Hum_pre_7","Hum_cont","Prec_pre_30","Prec_pre_15","Prec_pre_7","Prec_cont","remoteness"]].copy()
fires_df.head(10)

,fire_size,fire_size_class,state,Vegetation,Temp_pre_30,Temp_pre_15,Temp_pre_7,Temp_cont,Wind_pre_30,Wind_pre_15,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,10.0,C,PR,12,24.480974,24.716923,24.902597,24.527961,4.341807,3.492857,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,3.0,B,TN,15,7.553433,7.010000,0.343529,10.448298,2.709764,2.881707,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,60.0,C,MS,16,4.971930,5.782766,5.558750,13.696600,3.364499,2.923830,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,1.0,B,NV,0,16.275967,18.996181,18.142564,0.000000,4.054982,3.398329,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,2.0,B,LA,12,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633
5,1.0,B,GA,12,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.139643
6,5.2,B,GA,12,14.877341,16.409326,0.000000,0.000000,2.000214,1.727202,...,0.000000,79.896679,73.431818,0.000000,0.000000,26.0,0.0,0.0,0.0,0.148904
7,1.0,B,TX,12,16.851939,16.997783,20.434783,11.985560,1.331257,1.472949,...,2.148857,72.899478,75.061381,77.924623,70.732911,28.4,27.5,1.2,55.4,0.241894
8,1.0,B,AR,15,26.655241,27.264870,28.968064,28.682688,1.768074,1.705297,...,2.103090,68.319022,67.575419,65.077844,60.196858,6.6,3.3,0.0,46.4,0.224629
9,1.0,B,ND,15,4.600950,6.861878,6.053333,0.000000,6.380760,6.334254,...,0.000000,64.606509,55.943038,54.337838,0.000000,12.3,1.8,0.0,0.0,0.291683


##Data Preprocessing
We have now loaded our data into a Pandas dataframe with the columns we deemed useful for our analysis. Next we need to filter the 'state' column for 'CA' because we are only looking for Califronia wildfires. We also found an abundance of null data in the columns containing the weather readings, so next we filter those out as well. We are left with a dataframe of only California fires with usable weather data. Now we can use this data to train our model.

In [128]:
# Filter Out unusable data
fires_df = fires_df[fires_df["state"]=="CA"]
fires_df = fires_df[fires_df["Temp_pre_30"]>0]
fires_df = fires_df[fires_df["Temp_pre_15"]>0]
fires_df = fires_df[fires_df["Temp_pre_7"]>0]
fires_df = fires_df[fires_df["Temp_cont"]>0]
fires_df = fires_df[fires_df["Wind_pre_30"]>0]
fires_df = fires_df[fires_df["Wind_pre_15"]>0]
fires_df = fires_df[fires_df["Wind_pre_7"]>0]
fires_df = fires_df[fires_df["Wind_cont"]>0]
fires_df = fires_df[fires_df["Hum_pre_30"]>0]
fires_df = fires_df[fires_df["Hum_pre_15"]>0]
fires_df = fires_df[fires_df["Hum_pre_7"]>0]
fires_df = fires_df[fires_df["Hum_cont"]>0]
fires_df = fires_df[fires_df["Prec_pre_30"]>-1]
fires_df = fires_df[fires_df["Prec_pre_15"]>-1]
fires_df = fires_df[fires_df["Prec_pre_7"]>-1]
fires_df = fires_df[fires_df["Prec_cont"]>-1]

fires_df.head(10)

,fire_size,fire_size_class,state,Vegetation,Temp_pre_30,Temp_pre_15,Temp_pre_7,Temp_cont,Wind_pre_30,Wind_pre_15,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
78,450.0,E,CA,16,27.850483,28.165097,28.499405,33.900000,2.580000,2.664543,...,2.237500,44.706207,42.983379,49.000000,39.375000,0.0,0.0,0.0,0.0,0.472150
88,3.0,B,CA,15,19.685119,19.242174,19.495294,19.063744,3.893413,3.755263,...,3.751422,67.688935,68.160550,64.461538,66.811705,0.0,0.0,0.0,0.0,0.501750
192,1.0,B,CA,14,11.748991,13.265223,15.781065,14.750000,1.548318,1.497429,...,1.102083,61.600837,60.522481,58.560897,62.888889,2.5,2.5,0.0,0.0,0.501108
274,1.3,B,CA,16,18.601678,17.351402,17.417453,13.176536,1.935570,2.238318,...,2.894382,58.164848,52.572127,50.336788,68.926554,0.0,0.0,0.0,27.9,0.461236
302,5.0,B,CA,12,17.951613,17.726636,18.071066,19.908578,2.254147,2.099766,...,1.995372,73.609502,74.295508,73.903553,79.306030,2.9,1.5,0.6,2.1,0.466788
309,2.0,B,CA,0,4.610849,5.639908,7.474468,8.600308,1.652524,1.367890,...,2.131963,72.852975,70.987654,76.985714,78.735746,12.2,3.8,3.8,49.1,0.499333
394,1.0,B,CA,0,16.092461,19.914530,21.184049,23.756098,3.233091,3.683086,...,4.056860,57.811463,45.103343,34.831081,43.171200,29.2,0.0,0.0,0.0,0.501295
436,1.0,B,CA,0,26.329749,28.340288,30.689062,28.071181,2.667626,2.358993,...,2.127778,33.115108,29.446043,27.703125,41.237762,0.0,0.0,0.0,0.0,0.497625
442,2.0,B,CA,0,22.488692,23.002949,20.030556,16.693987,2.714033,2.576944,...,2.174346,51.244202,50.099462,60.648045,55.608639,12.0,12.0,12.0,8.8,0.499643
453,1.0,B,CA,14,23.426667,27.206389,25.695833,26.551489,3.217222,2.839167,...,3.064965,43.815385,39.416667,42.446429,39.747518,21.0,0.0,0.0,3.0,0.520804


## Model Choice and Changes
Originally, we chose the sklearn logistic regression binary classifier as our model because we wanted to focus solely on 'G' class fires. So, we used a lambda function to convert the 'fire_size_class' column to a binary code, with 'G' being equal to 1, and every other class being set to 0. We eventually converted the model to work with multiple classes so that it could predict any class 'B' through 'G' (since this dataset does not contain 'A' class fires). 

In [129]:
# Modify Fire size class column for binary classifier
# G = 1, Everything else = 0
#fires_df["fire_size_class"] = fires_df["fire_size_class"].apply(lambda x: 1 if x == 'G' else 0)


In [130]:
fires_df.head(10)

,fire_size,fire_size_class,state,Vegetation,Temp_pre_30,Temp_pre_15,Temp_pre_7,Temp_cont,Wind_pre_30,Wind_pre_15,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
78,450.0,E,CA,16,27.850483,28.165097,28.499405,33.900000,2.580000,2.664543,...,2.237500,44.706207,42.983379,49.000000,39.375000,0.0,0.0,0.0,0.0,0.472150
88,3.0,B,CA,15,19.685119,19.242174,19.495294,19.063744,3.893413,3.755263,...,3.751422,67.688935,68.160550,64.461538,66.811705,0.0,0.0,0.0,0.0,0.501750
192,1.0,B,CA,14,11.748991,13.265223,15.781065,14.750000,1.548318,1.497429,...,1.102083,61.600837,60.522481,58.560897,62.888889,2.5,2.5,0.0,0.0,0.501108
274,1.3,B,CA,16,18.601678,17.351402,17.417453,13.176536,1.935570,2.238318,...,2.894382,58.164848,52.572127,50.336788,68.926554,0.0,0.0,0.0,27.9,0.461236
302,5.0,B,CA,12,17.951613,17.726636,18.071066,19.908578,2.254147,2.099766,...,1.995372,73.609502,74.295508,73.903553,79.306030,2.9,1.5,0.6,2.1,0.466788
309,2.0,B,CA,0,4.610849,5.639908,7.474468,8.600308,1.652524,1.367890,...,2.131963,72.852975,70.987654,76.985714,78.735746,12.2,3.8,3.8,49.1,0.499333
394,1.0,B,CA,0,16.092461,19.914530,21.184049,23.756098,3.233091,3.683086,...,4.056860,57.811463,45.103343,34.831081,43.171200,29.2,0.0,0.0,0.0,0.501295
436,1.0,B,CA,0,26.329749,28.340288,30.689062,28.071181,2.667626,2.358993,...,2.127778,33.115108,29.446043,27.703125,41.237762,0.0,0.0,0.0,0.0,0.497625
442,2.0,B,CA,0,22.488692,23.002949,20.030556,16.693987,2.714033,2.576944,...,2.174346,51.244202,50.099462,60.648045,55.608639,12.0,12.0,12.0,8.8,0.499643
453,1.0,B,CA,14,23.426667,27.206389,25.695833,26.551489,3.217222,2.839167,...,3.064965,43.815385,39.416667,42.446429,39.747518,21.0,0.0,0.0,3.0,0.520804


## Model Training and Testing
We train the model on the cleaned data, omitting the 'fire_size', 'fire_size_class', and 'state' columns. We omit the 'fire_size' column because it directly coorelates with the fire_size_class.

In [131]:
# Separate the Features (X) from the Target (y)
y = fires_df["fire_size_class"]
X = fires_df.drop(columns=["fire_size_class","fire_size","state"])

In [132]:
# Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=5, 
                                                    stratify=y)
X_train.shape


(1357, 18)

In [133]:
# Create a Logistic Regression Model
classifier = LogisticRegression(solver='newton-cg',
                                max_iter=100,
                                multi_class='ovr',
                                random_state=5)

In [134]:
# Fit (train) or model using the training data
classifier.fit(X_train, y_train)

LogisticRegression(multi_class='ovr', random_state=5, solver='newton-cg')

In [135]:
# Make predictions
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,B,C
1,B,B
2,G,G
3,B,B
4,B,B
5,B,C
6,B,B
7,B,D
8,B,B
9,B,C


In [136]:
# Print the balanced accuracy score
print(accuracy_score(y_test, y_pred))

0.7549668874172185


In [137]:
# Print the confusion matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[284   0   0   0   0   0]
 [ 59   0   0   0   0   0]
 [ 13   0   0   0   0   0]
 [  7   0   0   0   0   0]
 [  4   0   0   0   0  23]
 [  3   0   0   0   2  58]]


In [138]:
# Print the classification report
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           B       0.77      1.00      0.87       284
           C       0.00      0.00      0.00        59
           D       0.00      0.00      0.00        13
           E       0.00      0.00      0.00         7
           F       0.00      0.00      0.00        27
           G       0.72      0.92      0.81        63

    accuracy                           0.75       453
   macro avg       0.25      0.32      0.28       453
weighted avg       0.58      0.75      0.66       453



/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [139]:
from sklearn.model_selection import GridSearchCV
sklearn_tuning = LogisticRegression(random_state=5)
param_grid = {
    'max_iter': [100, 200, 500],
    'multi_class': ['ovr', 'multinomial', 'auto'],
    'solver' :['newton-cg','lbfgs','saga']
    }
GSCV = GridSearchCV(estimator=sklearn_tuning, param_grid=param_grid, cv=5)
GSCV.fit(X_train, y_train)
print(GSCV.best_params_)

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached w

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharaz

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

...

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached w

/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/Users/remiharazim/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scik

{'max_iter': 100, 'multi_class': 'ovr', 'solver': 'newton-cg'}
